In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Greedy Matrix Codenames

A first attempt at the greedy matrix approach to playing codenames as a Spymaster. Essentially, all ally word combinations of up to 5 are calculated and clues are generated for each by using Gensim's word similarity functions. The resulting clue words are compared with the rest of the board and, using weightings for card types, scores are generated.

For this first pass, we're using the conceptnet model in the hope that it's most suited to the codenames task. I say hope because at this point it requires further work to evaluate the various models available (read as 'stab in the dark').

## Helper Methods and Prep

In [2]:
from os import getcwd, chdir

chdir("../")  # Hate this, but quick and dirty
import pandas as pd
import numpy as np
import requests
import zipfile
import plotly.express as px
import plotly.graph_objs as go
import io
import gensim.downloader as api
import logging

from collections import namedtuple
from sklearn.metrics.pairwise import euclidean_distances
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from itertools import combinations
from codenames.codenames_board import CodenamesGame
from tqdm.auto import tqdm
from os.path import isfile, join

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

<IPython.core.display.Javascript object>

In [191]:
model_name = "conceptnet-numberbatch-17-06-300"
model_path = join('models', model_name)
if isfile(model_path):
    model = KeyedVectors.load(model_path)
else:
    model = api.load(
        model_name
    )
    model.save(model_path)


<IPython.core.display.Javascript object>

In [192]:
# A small adjustment to the vocab to remove all none english words and strip away the language prefix, can also be used in the future to limit vocab for the guessing agent
(
    english_only_vocab,
    english_only_vectors,
    english_only_index2entity,
    english_only_vectors_norm,
) = ({}, [], [], [])

for word, vector, index2entity in zip(
    model.vocab.keys(), model.vectors, model.index2entity
):

    if "/c/en/" in word:
        vocab = model.vocab[word]
        
        vocab.index = len(english_only_index2entity)
        english_only_vocab[word.replace("/c/en/","")] = vocab
        english_only_vectors.append(vector)
        english_only_index2entity.append(index2entity.replace("/c/en/",""))

model.vocab = english_only_vocab
model.vectors = np.array(english_only_vectors)
model.index2entity = english_only_index2entity
model.index2word = english_only_index2entity


<IPython.core.display.Javascript object>

In [257]:
[1, 2, 3][:1]

[1]

<IPython.core.display.Javascript object>

In [267]:
def get_game_data(codenames_game, model):
    team = codenames_game.current_team
    enemy_team = "blue" if team == "red" else "blue"
    spymaster_map = codenames_game.spymaster_map
    cards = [smart_lower(card, model) for card in codenames_game.codename_cards]
    ally_cards = [c for i, c in enumerate(cards) if i in spymaster_map[team]]
    ally_cards = [smart_lower(ally_card, model) for ally_card in ally_cards]
    enemy_cards = [c for i, c in enumerate(cards) if i in spymaster_map[enemy_team]]
    enemy_cards = [smart_lower(enemy_card, model) for enemy_card in enemy_cards]
    assassin_card = cards[spymaster_map["assassin"][0]]

    return cards, ally_cards, enemy_cards, assassin_card, spymaster_map, team


def smart_lower(word, model):
    checked = False
    adjusted = word.replace(" ", "").lower()
    try:
        model.get_vector(adjusted)
    except KeyError:
        adjusted = adjusted.capitalize()
        model.get_vector(adjusted)

    return adjusted


def create_word_combinations_matrices(ally_cards, model, default_max_combo=5):
    max_combination = min(len(ally_cards), default_max_combo)
    ally_combinations = []
    for i in range(1, max_combination + 1):
        ally_combinations += list(combinations(ally_cards, i))

    ally_combination_vectors = [
        [model.get_vector(word) for word in combination]
        for combination in ally_combinations
    ]
    return ally_combinations, ally_combination_vectors


def flatten_list_of_list(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]


def get_most_similar_vectors_for_combos(word_combinations, negative_cards=[], topn=1):
    most_similar_words = {
        combination: get_most_similar(
            positive_cards=list(combination), negative_cards=negative_cards, topn=topn
        )
        for combination in word_combinations
    }

    return most_similar_words


def get_most_similar(positive_cards, negative_cards=None, include_score=False, topn=1):
    try:
        return [
            (most_similar if include_score else most_similar[0])
            for most_similar in model.most_similar(
                positive=positive_cards, negative=negative_cards, topn=50,
            )
            if all([c.lower() not in most_similar[0].lower() for c in positive_cards])
        ][:topn]
    except IndexError:
        return [
            (most_similar if include_score else most_similar[0])
            for most_similar in model.most_similar(
                positive=positive_cards, negative=negative_cards, topn=100,
            )
            if all([c.lower() not in most_similar[0].lower() for c in positive_cards])
        ][:topn]


ClueTuple = namedtuple("ClueTuple", ["clue", "intended_combo", "board_similarities"])


def create_clue_tuples(word_combo_clue_dict, model):
    clue_tuples = []
    for clue_words, clues in word_combo_clue_dict.items():
        for clue in clues:
            clue_tuples.append(
                ClueTuple(
                    clue,
                    clue_words,
                    [(card, model.similarity(card, clue)) for card in cards],
                )
            )

    return clue_tuples


def create_clue_df(clue_tuples, cards):
    dataframe_tuples = []
    for clue, intended_combo, board_similarities in clue_tuples:
        dataframe_tuples.append(
            (
                clue,
                intended_combo,
                *[card_similarity[1] for card_similarity in board_similarities],
            )
        )

    return pd.DataFrame(
        dataframe_tuples, columns=["clue", "intended_combo"] + cards
    ).set_index(["clue", "intended_combo"])


def calculate_best_clue(clue_df, spymaster_map, enemy_colour, team_colour, **kwargs):
    assassin_weight = kwargs.get("assassin_weight", -10)
    enemy_weight = kwargs.get("enemy_weight", -5)
    neutral_weight = kwargs.get("neutral_weight", 0)
    ally_weight = kwargs.get("ally_weight", 10)
    risk_weight = kwargs.get("risk_weight", 0)
    clue_score_threshold = kwargs.get("clue_score_threshold", 0)
    ally_cards_len = len(spymaster_map[team_colour])
    weighted_clue_df = clue_df.copy()
    weighted_clue_df["raw_clue_length"] = weighted_clue_df.index.get_level_values(
        "intended_combo"
    ).str.len()
    # clip scores below the threshold
    weighted_clue_df.iloc[:, spymaster_map[team_colour]] = (
        ally_df := weighted_clue_df.iloc[:, spymaster_map[team_colour]]
    ).where(ally_df >= clue_score_threshold, 0)
    # If clue word score is part of intended, but below threshold we reduce the clue length!
    weighted_clue_df["zero_values"] = weighted_clue_df.apply(
        pd.Series.value_counts, axis=1
    ).loc[:, 0]
    weighted_clue_df["amended_combo"] = create_amended_combos(weighted_clue_df)
    weighted_clue_df["actual_combo_length"] = weighted_clue_df.amended_combo.str.len()

    # Apply various weights
    weighted_clue_df.iloc[:, spymaster_map["assassin"]] = (
        weighted_clue_df.iloc[:, spymaster_map["assassin"]] * assassin_weight
    )
    weighted_clue_df.iloc[:, spymaster_map["neutral"]] = (
        weighted_clue_df.iloc[:, spymaster_map["neutral"]] * neutral_weight
    )
    weighted_clue_df.iloc[:, spymaster_map[enemy_colour]] = (
        weighted_clue_df.iloc[:, spymaster_map[enemy_colour]] * enemy_weight
    )
    weighted_clue_df.iloc[:, spymaster_map[team_colour]] = (
        weighted_clue_df.iloc[:, spymaster_map[team_colour]] * ally_weight
    )

    weighted_clue_df["weighted_score"] = weighted_clue_df.iloc[:, :25].sum(axis=1)
    weighted_clue_df["weighted_score"] = weighted_clue_df.weighted_score + (
        weighted_clue_df.actual_combo_length * risk_weight
    )
    best_clue = weighted_clue_df.iloc[weighted_clue_df["weighted_score"].argmax()]

    return best_clue, weighted_clue_df


def create_amended_combos(df):
    #  Can be done with apply, doing it dirtily in the hope of a heavy refactor after the fact
    new_combos = []
    for idx, row in df.iterrows():
        intended_combo = idx[1]
        zero_values = row.loc[row == 0]
        new_combo = [c for c in intended_combo if c not in zero_values.index]
        new_combos.append(new_combo)
    return new_combos

<IPython.core.display.Javascript object>

## Walkthrough

In [49]:
codenames_game = CodenamesGame(words_loc="codenames/words.txt")
cards, ally_cards, enemy_cards, assassin_card, spymaster_map, team = get_game_data(
    codenames_game, model
)
enemy_team = "blue" if team == "red" else "red"

<IPython.core.display.Javascript object>

In [50]:
word_combos, vector_combos = create_word_combinations_matrices(ally_cards, model)
clues = get_most_similar_vectors_for_combos(word_combos, [assassin_card])
clue_tuples = create_clue_tuples(clues, word_combos, model)

clue_df = create_clue_df(clue_tuples, cards)

best_clue, weighted_df = calculate_best_clue(
    clue_df,
    spymaster_map,
    enemy_colour=enemy_team,
    team_colour=team,
    assassin_weight=-10,
    enemy_weight=-1,
    neutral_weight=-1,
    ally_weight=1,
    risk_weight=0.3,
    clue_score_threshold=0.2,
)
# Simply removing the words from the intended combos that fall below the threshold
weighted_df["amended_combo"] = create_amended_combos(weighted_df)
weighted_df["amended_combo"] = weighted_df.amended_combo.astype("object")

weighted_df = weighted_df[weighted_df.actual_combo_length >= 1]
# Removing any clue duplicates (where there were multiple intended combos but the amended combos are identical)
weighted_df = (
    weighted_df.reset_index().drop_duplicates(subset=["clue", "weighted_score"]).sort_values(['weighted_score'], ascending=False)
)
# Column reorder for readbility
weighted_df = weighted_df[list(weighted_df.columns[:2]) + list(weighted_df.columns[27:]) + list(
    weighted_df.columns[2:27]
)]


<IPython.core.display.Javascript object>

In [124]:
weighted_df.style

,clue,intended_combo,raw_clue_length,zero_values,actual_combo_length,weighted_score,amended_combo,kiwi,dragon,hand,bond,suit,boom,card,conductor,paste,litter,play,atlantis,fire,square,duck,icecream,superhero,genius,whale,cloak,sound,casino,gold,london,grass
4,cacolet,"('litter',)",1,8.000000,1.000000,2.622437,['litter'],0.000000,0.000000,0.087829,0.039508,0.120533,0.042767,0.061511,0.000000,0.000000,0.869181,0.055569,0.034587,0.016408,0.021054,0.074013,0.036035,0.073470,0.000000,0.000000,0.097255,0.625308,0.030329,0.037082,0.000000,0.000000
223,clumper,"('conductor', 'paste', 'litter', 'grass')",4,7.000000,2.000000,2.287334,"['litter', 'grass']",0.000000,0.000000,-0.015225,0.002136,0.067822,0.001282,0.015473,0.000000,0.000000,0.237867,-0.033566,0.064963,-0.040425,-0.090546,0.054047,0.032726,0.143695,0.000000,0.000000,-0.000445,0.493775,-0.032648,0.069373,0.000000,0.717029
173,karengo,"('kiwi', 'paste', 'whale', 'grass')",4,5.000000,4.000000,2.244346,"['kiwi', 'paste', 'whale', 'grass']",0.460699,0.000000,0.040467,0.000484,-0.000646,-0.035990,0.000299,0.000000,0.280808,0.000000,0.082966,-0.023984,0.069587,0.021224,-0.054183,-0.211370,-0.089774,0.000000,0.228519,-0.032509,0.100308,0.055458,-0.087855,0.000000,0.239836
44,yorkshire_fog,"('london', 'grass')",2,8.000000,1.000000,1.692649,['grass'],0.000000,0.000000,0.054791,0.017756,0.053260,0.099613,0.019541,0.000000,0.000000,0.000000,-0.058310,0.009146,-0.060063,-0.030157,-0.009628,-0.010287,0.096254,0.000000,0.000000,0.020522,0.220712,0.032421,0.015197,0.000000,0.921881
62,kokihi,"('kiwi', 'paste', 'grass')",3,7.000000,2.000000,1.515733,"['kiwi', 'grass']",0.367290,0.000000,0.014648,-0.046188,0.043977,-0.010587,-0.087318,0.000000,0.000000,0.000000,0.035394,0.032025,-0.026256,-0.094456,-0.040773,-0.090947,-0.067398,0.000000,0.000000,-0.076818,0.544034,0.071026,-0.177865,0.000000,0.525945
8,munj,"('grass',)",1,8.000000,1.000000,1.371186,['grass'],0.000000,0.000000,0.014687,-0.006860,0.058932,0.075021,-0.021201,0.000000,0.000000,0.000000,-0.083766,0.007894,-0.100503,-0.059613,-0.020658,0.010771,0.089783,0.000000,0.000000,0.010441,0.071537,0.015164,0.020963,0.000000,0.988593
3,computer_command,"('paste',)",1,8.000000,1.000000,1.337864,['paste'],0.000000,0.000000,-0.028187,0.007796,0.049073,-0.018391,0.006823,0.000000,0.732278,0.000000,0.114337,0.031593,0.058971,-0.001835,-0.047076,-0.041338,0.017985,0.000000,0.000000,-0.008984,0.087886,0.090940,-0.014009,0.000000,0.000000
15,heathrow_injection,"('kiwi', 'london')",2,7.000000,2.000000,1.281000,"['kiwi', 'london']",0.407381,0.000000,0.009191,-0.010305,-0.026554,-0.041185,0.002262,0.000000,0.000000,0.000000,0.035760,-0.114248,0.022178,-0.055922,-0.032522,-0.145580,-0.079145,0.000000,0.000000,-0.012761,0.239989,0.005067,-0.088744,0.566140,0.000000
0,chinese_gooseberry,"('kiwi',)",1,8.000000,1.000000,1.108292,['kiwi'],0.976367,0.000000,0.042295,-0.062691,0.073874,-0.049519,0.011383,0.000000,0.000000,0.000000,0.040176,0.006787,0.064327,0.076206,-0.030650,-0.243317,-0.122000,0.000000,0.000000,0.077498,0.066875,-0.008153,-0.111165,0.000000,0.000000
65,enzedder,"('kiwi', 'litter', 'london')",3,8.000000,1.000000,1.102157,['kiwi'],0.931236,0.000000,0.042304,-0.027898,0.083989,-0.055649,0.030716,0.000000,0.000000,0.000000,0.051314,-0.032011,0.070143,0.093443,-0.017240,-0.200496,-0.121196,0.000000,0.000000,0.044972,-0.013487,-0.014725,-0.063259,0.000000,0.000000


<IPython.core.display.Javascript object>

Interesting clues come out of this, but it's obvious that the vocabulary is a little too good. We have a list of the 100k most used words in the english vocab, using this we can reduce it down somewhat. Initially, we can just take 100k and see the effect. After this there may be some room to experiment with taking different slices or randomly selecting. It should be noted that for the guessers, this will be refined, making the guessers randomly articulate.

In [189]:
def articulate_vocab_with_set(model, vocab_set):
    (
        english_only_vocab,
        english_only_vectors,
        english_only_index2entity,
        english_only_vectors_norm,
    ) = ({}, [], [], [])

    for word, vector, index2entity in zip(
        model.vocab.keys(), model.vectors, model.index2entity
    ):

        if word in vocab_set:
            vocab = model.vocab[word]
            vocab.index = len(english_only_index2entity)
            english_only_vocab[word] = vocab
            english_only_vectors.append(vector)
            english_only_index2entity.append(index2entity)

    model.vocab = english_only_vocab
    model.vectors = np.array(english_only_vectors)
    model.index2entity = english_only_index2entity
    model.index2word = english_only_index2entity

    return model

<IPython.core.display.Javascript object>

In [190]:
vocab_lines = open("data/en_vocab_100k.txt", "r").readlines()
vocab_list = [l.replace("\n", "").lower() for l in vocab_lines if "#!" not in l]
# Need to remember codenames words!!
with open('codenames/words.txt', "r", newline="\n") as inputfile:
    codenames_words = inputfile.read().split("\n")
vocab_list += list(map(str.lower, codenames_words))
vocab_set = set(vocab_list)


<IPython.core.display.Javascript object>

In [193]:
fully_artiulcated_vocab = len(model.index2entity)
model = articulate_vocab_with_set(model, vocab_set)
# To get vocab size
fully_artiulcated_vocab, len(model.index2entity)

(417194, 46325)

<IPython.core.display.Javascript object>

We have a much reduced vocab size - around 10% of the fully articulate model - so the results in the spymaster clue df will presumably be of a lower clue size.

In [286]:
word_combos, vector_combos = create_word_combinations_matrices(ally_cards, model)
word_combo_clue_dict = get_most_similar_vectors_for_combos(
    word_combos, [assassin_card], topn=3
)
clue_tuples = create_clue_tuples(word_combo_clue_dict, model)

clue_df = create_clue_df(clue_tuples, cards)

best_clue, weighted_df = calculate_best_clue(
    clue_df,
    spymaster_map,
    enemy_colour=enemy_team,
    team_colour=team,
    assassin_weight=-10,
    enemy_weight=-5,
    neutral_weight=-1,
    ally_weight=1,
    risk_weight=1,
    clue_score_threshold=0.1,
)

weighted_df = weighted_df[weighted_df.actual_combo_length >= 1]
# Removing any clue duplicates (where there were multiple intended combos but the amended combos are identical)
weighted_df = (
    weighted_df.reset_index()
    .drop_duplicates(subset=["clue", "weighted_score", "actual_combo_length"],)
    .sort_values(["weighted_score"], ascending=False)
)

weighted_df.reset_index(inplace=True)
# Column reorder for readbility
weighted_df = weighted_df[
    list(weighted_df.columns[:2])
    + list(weighted_df.columns[27:])
    + list(weighted_df.columns[2:27])
]
weighted_df.style

,index,clue,grass,raw_clue_length,zero_values,amended_combo,actual_combo_length,weighted_score,intended_combo,kiwi,dragon,hand,bond,suit,boom,card,conductor,paste,litter,play,atlantis,fire,square,duck,icecream,superhero,genius,whale,cloak,sound,casino,gold,london
0,917,undergrowth,0.414210,5,6.000000,"['kiwi', 'litter', 'grass']",3,5.901493,"('kiwi', 'conductor', 'litter', 'london', 'grass')",0.138103,0.000000,0.036583,-0.142332,0.578287,-0.025072,0.035667,0.000000,0.000000,0.244974,0.325496,0.371985,-0.127119,0.177184,0.008986,0.374288,0.410117,0.000000,0.000000,-0.113965,0.030731,0.104185,0.059184,0.000000
1,778,radish,0.289630,5,5.000000,"['kiwi', 'dragon', 'paste', 'grass']",4,5.596839,"('kiwi', 'dragon', 'conductor', 'paste', 'grass')",0.126621,0.138908,0.083841,0.317496,0.077784,0.011510,-0.031775,0.000000,0.229054,0.000000,0.121842,0.177084,0.054691,0.011252,-0.076343,-0.547777,0.142656,0.000000,0.000000,0.022022,0.406355,0.032585,0.009404,0.000000
2,834,seaweed,0.295075,5,4.000000,"['kiwi', 'dragon', 'paste', 'whale', 'grass']",5,5.434232,"('kiwi', 'dragon', 'paste', 'whale', 'grass')",0.116152,0.180823,0.017932,0.144891,-0.053942,0.012072,-0.036551,0.000000,0.216606,0.000000,0.283673,-0.173864,0.068080,-0.069953,-0.082803,-0.657073,-0.076695,0.000000,0.351802,-0.037495,-0.160203,0.046414,0.049293,0.000000
3,625,kelp,0.271682,4,5.000000,"['dragon', 'paste', 'whale', 'grass']",4,5.324098,"('dragon', 'paste', 'whale', 'grass')",0.000000,0.192682,0.084415,0.368943,0.083186,-0.012050,0.236247,0.000000,0.183941,0.000000,0.541860,-0.167500,0.055832,-0.034236,-0.097044,-0.261275,0.070408,0.000000,0.383216,-0.011393,-0.698873,0.098575,0.035481,0.000000
4,336,herbage,0.754343,3,6.000000,"['paste', 'litter', 'grass']",3,5.213207,"('paste', 'litter', 'grass')",0.000000,0.000000,0.054162,0.025751,0.384944,0.055647,-0.014643,0.000000,0.110493,0.154521,0.080584,0.446377,-0.039344,-0.145151,-0.051821,-0.205465,0.461842,0.000000,0.000000,-0.017765,0.147564,0.025392,-0.014225,0.000000
5,890,prune,0.254166,5,5.000000,"['kiwi', 'paste', 'grass']",3,5.129393,"('kiwi', 'conductor', 'paste', 'genius', 'grass')",0.254175,0.000000,0.005939,-0.113117,0.071895,0.012577,0.168429,0.000000,0.193534,0.112231,-0.002549,0.488500,-0.015513,-0.219028,0.012731,-0.257771,0.352893,0.000000,0.000000,0.033117,0.680530,0.092999,0.003654,0.000000
6,569,radish,0.289630,4,5.000000,"['dragon', 'paste', 'grass']",3,4.596839,"('dragon', 'conductor', 'paste', 'grass')",0.126621,0.138908,0.083841,0.317496,0.077784,0.011510,-0.031775,0.000000,0.229054,0.000000,0.121842,0.177084,0.054691,0.011252,-0.076343,-0.547777,0.142656,0.000000,0.000000,0.022022,0.406355,0.032585,0.009404,0.000000
7,537,vegetation,0.559784,4,6.000000,"['kiwi', 'litter', 'grass']",3,4.541062,"('kiwi', 'litter', 'whale', 'grass')",0.151573,0.000000,0.031048,0.034995,0.120751,-0.002553,-0.064906,0.000000,0.000000,0.224165,0.182539,-0.077653,-0.131374,-0.087651,-0.015906,0.060403,0.506510,0.000000,0.000000,-0.019308,0.038664,0.003722,0.026259,0.000000
8,445,seaweed,0.295075,4,4.000000,"['kiwi', 'dragon', 'whale', 'grass']",4,4.434232,"('kiwi', 'dragon', 'whale', 'grass')",0.116152,0.180823,0.017932,0.144891,-0.053942,0.012072,-0.036551,0.000000,0.216606,0.000000,0.283673,-0.173864,0.068080,-0.069953,-0.082803,-0.657073,-0.076695,0.000000,0.351802,-0.037495,-0.160203,0.046414,0.049293,0.000000
9,446,kelp,0.271682,4,5.000000,"['dragon', 'whale', 'grass']",3,4.324098,"('kiwi', 'dragon', 'whale', 'grass')",0.000000,0.192682,0.084415,0.368943,0.083186,-0.012050,0.236247,0.000000,0.183941,0.000000,0.541860,-0.167500,0.055832,-0.034236,-0.097044,-0.261275,0.070408,0.000000,0.383216,-0.011393,-0.698873,0.098575,0.035481,0.000000


<IPython.core.display.Javascript object>